In [1]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [2]:
df_train = pd.read_csv('D:\\YNOV\\M1\\NLP\\Projet\\cnn_dailymail\\train.csv').sample(frac=0.1, random_state=42).reset_index(drop=True)
df_test = pd.read_csv('D:\\YNOV\\M1\\NLP\\Projet\\cnn_dailymail\\test.csv').sample(frac=0.1, random_state=42).reset_index(drop=True)
df_validation = pd.read_csv('D:\\YNOV\\M1\\NLP\\Projet\\cnn_dailymail\\validation.csv').sample(frac=0.1, random_state=42).reset_index(drop=True)

In [4]:
print(df_train['article'].str.len().min()) 
print(df_train['highlights'].str.len().min()) 

117
37


In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def clean_text(text):   #no complete clean of punctuation to keep valid number/name     U.S | 10/02/2015 
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\w*\d\w*', '', text)    
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(tokens)

def clean_highlight(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    return ' '.join(tokens)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rapha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rapha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df_train['clean_article'] = df_train['article'].apply(clean_text)
df_train['clean_summary'] = df_train['highlights'].apply(clean_highlight)

df_test['clean_article'] = df_test['article'].apply(clean_text)
df_test['clean_summary'] = df_test['highlights'].apply(clean_highlight)

df_validation['clean_article'] = df_validation['article'].apply(clean_text)
df_validation['clean_summary'] = df_validation['highlights'].apply(clean_highlight)

In [5]:
sentences = df_train['clean_article'].apply(lambda x: x.split()).tolist()

w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

w2v_model.save("word2vec.model")

In [6]:
def vectorize_sentence(sentence, model, max_len=300):
    words = sentence.split()
    vectorized = []
    for word in words[:max_len]:
        if word in model.wv:
            vectorized.append(model.wv[word])
        else:
            vectorized.append(np.zeros(model.vector_size))  
    while len(vectorized) < max_len:
        vectorized.append(np.zeros(model.vector_size))
    return np.array(vectorized, dtype=np.float32)

X_train = np.array([vectorize_sentence(text, w2v_model) for text in df_train['clean_article']])
y_train = df_train['clean_summary'].values  

In [7]:
latent_dim = 256
timesteps = 300
embedding_dim = 100

encoder_inputs = Input(shape=(timesteps, embedding_dim))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(timesteps, embedding_dim)) 
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(w2v_model.vector_size, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='mse')

In [8]:
def vectorize_sentence_shifted(sentence, model, max_len=300):
    words = sentence.split()
    vectorized = []
    for word in words[:max_len]:
        if word in model.wv:
            vectorized.append(model.wv[word])
        else:
            vectorized.append(np.zeros(model.vector_size))  
    while len(vectorized) < max_len:
        vectorized.append(np.zeros(model.vector_size))
    return np.array(vectorized, dtype=np.float32)

X_train = np.array([vectorize_sentence(text, w2v_model) for text in df_train['clean_article']])

# decoder with temporal sift
X_decoder_input = []
Y_decoder_output = []

for summary in df_train['clean_summary']:
    full_vec = vectorize_sentence_shifted(summary, w2v_model)

    decoder_input = full_vec[:-1]  
    decoder_output = full_vec[1:]  

    pad = np.zeros((1, w2v_model.vector_size), dtype=np.float32)
    decoder_input = np.vstack([decoder_input, pad])      
    decoder_output = np.vstack([decoder_output, pad])    

    X_decoder_input.append(decoder_input)
    Y_decoder_output.append(decoder_output)

X_decoder_input = np.array(X_decoder_input)
Y_decoder_output = np.array(Y_decoder_output)

In [9]:
model.fit(
    [X_train, X_decoder_input],
    Y_decoder_output,
    batch_size=32,
    epochs=2,
    validation_split=0.1
)


Epoch 1/2
808/808 ━━━━━━━━━━━━━━━━━━━━ 4177s 5s/step - loss: 0.1287 - val_loss: 0.1257
Epoch 2/2
808/808 ━━━━━━━━━━━━━━━━━━━━ 3380s 4s/step - loss: 0.1237 - val_loss: 0.1244


In [ ]:
model.save("Resume_Word2Vec_Model.keras")

In [14]:
X_test = np.array([vectorize_sentence(text, w2v_model) for text in df_test['clean_summary']])
X_decoder_test = np.array([vectorize_sentence(text, w2v_model) for text in df_test['clean_summary']])
Y_decoder_test = np.array([vectorize_sentence(text, w2v_model) for text in df_test['clean_summary']])

loss = model.evaluate([X_test, X_decoder_test], Y_decoder_test, batch_size=32)
print("Test MSE Loss:", loss)

36/36 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - loss: 0.1310
Test MSE Loss: 0.1328573077917099


In [15]:
encoder_model = Model(encoder_inputs, encoder_states)

In [16]:
# Inputs for decoder inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_inf = Input(shape=(1, embedding_dim))  # One time step at a time

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs_inf, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs_inf] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [21]:
encoder_model.save("encoder_model.keras")
decoder_model.save("decoder_model.keras")

In [17]:
def decode_sequence(input_seq, max_len=30):

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, embedding_dim))

    decoded_sentence = []

    for _ in range(max_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        word_vector = output_tokens[0, -1, :]
        similar_words = w2v_model.wv.similar_by_vector(word_vector, topn=8) #topn = 10 To fix the "server server server" problem
        sampled_word = np.random.choice([w for w, _ in similar_words])
        decoded_sentence.append(sampled_word)

        target_seq[0, 0, :] = w2v_model.wv[sampled_word]

        states_value = [h, c]

    return ' '.join(decoded_sentence)


In [20]:
text = "The devastated parents of a nursing student whose body was found on a farm north of Glasgow have joined hundreds of mourners at a vigil. Karen Buckley vanished after a night out in the city at the weekend, sparking a four-day police search which yesterday found her body at High Craigton Farm. About 300 people - including the 24-year-old's mother and father Marian and John, and their son Kieran - attended a memorial at Glasgow's George Square today. A man, believed to be 21-year-old Alexander Pacteau, has been arrested in connection with Miss Buckley's death. He made a brief appearance in a private hearing at Glasgow Sheriff Court today. Scroll down for video . Grief stricken Marian and John Buckley (right) today joined hundreds of mourners at a vigil held in memory of their daughter Karen. Their son Kieran is pictured left . Miss Buckley was found dead on a farm north of Glasgow after going missing from a nightclub in the city . Miss Buckley's parents Marian and John (pictured) and two of her brothers arrived in George Square and looked at the tributes . A security van arrives at Glasgow Sheriff Court ahead of a court appearance by a 21-year-old charged with Miss Buckley's murder . Tearful friends, family and colleagues of the popular student laid flowers and left messages of condolence . At the vigil this afternoon, many brought flowers and candles to lay in the square.Â Cards and messages were also left, with people expressing their grief. A piper played as mourners laid flowers and others sang Scottish and Irish songs before a minute's silence was held. Towards the end of the vigil, Miss Buckley's parents Marian and JohnÂ and two of her brothers arrived in George Square and looked at the tributes. Marian Buckley hugged husband John as they read messages of sorrow and support for the family. Glasgow Caledonian University (GCU) - where Miss Buckley was studying occupational health - was well represented, with many young people in attendance wearing GCU tracksuits and jumpers. The vigil was organised by former Scottish Socialist politician Rosie Kane. She said: 'I don't want to be here, that's the first thing I need to say. Kieran Buckley, Karen's brother, is comforted by one of the many people who attended the vigil today . Karen's grief stricken mother Marian (pictured in the white and blue coat) embraces another mourner at the memorial . Show of support:Â Around 300 people gathered for a vigil in Glasgow today in memory of 24-year-old Karen Buckley . Four women embrace in George Square where hundreds of mourners gathered today for a silent vigil . 'It seems that everyone has been devastated but we want to support Karen's family and show them the best side of the city. We're trying to make something beautiful from that horrible, horrible grief. 'There have been some songs from Scotland and Ireland and people have brought flowers and candles. 'We also had a moment's silence to think about Karen's life, what we knew of her, about her family and to wish them the best.' The case has touched many people in Scotland and Ireland, with services held in Miss Buckley's home town of Cork earlier this week. Barry Ahearn, who is originally from Cork but now lives in Scotland, travelled from his home in Ayr to pay his respects to Miss Buckley in George Square. As organisers appealed for anyone who wanted to address the large crowd, Mr Ahearn sang a local Cork song. He said: 'I've been living in Scotland for the last four years and wanted to come today to show solidarity and support for Karen's family. A tearful woman is comforted by a man as crowds gathered to remember the nursing student . Karen Buckley vanished after a night out in the city at the weekend, sparking a four-day police search . A woman crouches down next to some of the many flowers and messages left in George Square during the vigil . A piper played as mourners laid flowers and others sang Scottish and Irish songs before a minute's silence was held . 'I sang On The Banks On My Own Lovely Lee - it's the river that runs through Cork and is the local anthem. 'The whole situation is so sad, particularly when you're living in Scotland and something like this happens to someone from your own country. It just doesn't fit, it's so sad. His wife Maura said: 'It has really touched us this week, it's so sad and we all feel it very much. 'There's a great number of people here tonight who I'm sure feel the same and we just want to rally round the Buckley family.' Flowers and messages have also been left at the GCU campus and at the entrance to High Craigton Farm, to the north of Glasgow, where the student's body was found. Flowers, cards and messages were also left as people expressed their grief during the vigil today . Tearful mourners huddled together as they paid their respects at the vigil today . Two women wipe away tears as they take part in the memorial. Ms Buckley moved to Scotland in February and was a first-year occupational therapy student at Glasgow Caledonian University . Politicians took time out of campaigning earlier today to offer their condolences to the Buckley family. Scotland's First Minister Nicola Sturgeon said: 'My thoughts are very much with the family and friends of Karen Buckley.' Jim Murphy began Labour's Scottish manifesto launch in Glasgow by saying: 'My thoughts and the thoughts of everyone in the room go out to Karen's family. None of us can in anyway imagine the pain and horror that they are going through. 'Glasgow and Scotland stands with you.' Former colleagues and friends in her hometown in County Cork, Ireland today remembered the 'beautiful, smiling' 24-year-old who was dedicated to helping the injured and sick. Her father, John, said yesterday that he, Miss Buckley's mother Marian and her brothers Brendan, Kieran, and Damian are 'absolutely heartbroken'. Friends and family have paid tribute to Karen Buckley - pictured with her brothers (l-r) Brendan, Damien and Kieran - after a four-day police search found her body yesterday . Her body was found at High Craigton farm after a huge search was launched to find her earlier this week . Miss Buckley's former supervisor at the Hibernian Hotel in the town of Mallow told the Irish Independent: 'She was always smiling, no matter how hard the work. She could never do enough to help you.' Karenâ€™s former  school principal, Susan Oâ€™Keefe, said: â€˜She was a beautiful girl and she was a great pupil. 'Itâ€™s sad. We are here on a fine summerâ€™s day but Karen is not. Weâ€™re all just the worse off for such a lovely person been taken from us, and for everyone in this community, it is a shocking tragedy.â€™ Karen was a â€˜great all-rounderâ€™, participating in ladiesâ€™ football with Clyda Rovers and athletics with her school in Analeetha, Ms Oâ€™Keefe added. â€˜At that time we were a much smaller school then we are now, so we were just like a big family. It is a big blow.' Yvonne Bane, the principal at Miss Buckley's former school in Mallow added: 'The community in St Mary's Secondary School is united in grief at the sad news concerning Karen Buckley, a former student of our school who completed her Leaving Certificate here in 2009. 'A prayer service took place in the school this morning, with all students and staff of St. Mary's Secondary School in attendance. We offered prayers for Karen and her family at this difficult time. Police initially searched a park in Glasgow, but attention later switched to the farm six miles from her home . Floral tributes were left at the scene after her family told of their 'absolute heartbreak' at her death . A vigil will be held in Glasgow today as friends and family tell of their shock at this week's events . 'The school community have been sharing warm memories of our beautiful, former student over the last number of days.' And a spokeswoman for the University of Limerick, where Karen studied nursing, said: 'Karen was a very valued member of the UL Community as well as the wider nursing and medical communities in Limerick, having spent time working in both the University Hospital Limerick and the University Maternity Hospital. 'The thoughts of the entire UL community are with her parents and family and her wide circle of friends as well as her UL classmates and medical colleagues.' A 21-year-old man was arrested in connection with Ms Buckley's death yesterday. He made a brief appearance in a private hearing at Glasgow Sheriff Court today. Friends of Ms Buckley raised the alarm on Sunday morning when she failed to return from a night out at The Sanctuary in the west end of Glasgow. The search led police to Dawsholm Park, where Ms Buckley's handbag was found on Tuesday, and a property in the city's Dorchester Avenue, before the farm where the discovery was made. Ms Buckley moved to Scotland in February and was a first-year occupational therapy student at Glasgow Caledonian University. Miss Buckley's parents, Marian and John, came from Ireland to help the search for their 'cherished' daughter . Miss Buckley grew up in the town of Mallow, County Cork, whose residents expressed shock at her death . Her family, who travelled to Scotland on Tuesday, have said they are 'absolutely heartbroken'. Her father John Buckley, 62, said she was an outgoing girl who 'thoroughly enjoyed life'. He released a statement through Police Scotland which read: 'Marian and I, together with our sons Brendan, Kieran, and Damian are absolutely heartbroken. Karen was our only daughter, cherished by her family and loved by her friends. 'She was an outgoing girl who travelled the world, where she met lots of people and thoroughly enjoyed her life. We will miss her terribly.' More than Â£50,000 has been raised to support the Buckley family in an online drive organised by her former classmates at Limerick University. The 24-hour appeal closed last night after reaching Â£50,001 - ten times the initial target of Â£5,000. The Go Fund Me page was set up to cover the familys travel and expenses while in Scotland and to support the upkeep of their farm in Cork. Karen went missing from the Sanctuary nightclub not far from her home. Police initially searched a park in north-west Glasgow before switching their attention to a golf course and farm outside the city . Sorry we are not currently accepting comments on this article."
print("BBC TEXT : \n" + text)
text = clean_text(text)
sample_input = vectorize_sentence(text, w2v_model).reshape(1, 300, 100)
print("Resumer : ")
print(decode_sequence(sample_input))

BBC TEXT : 
The devastated parents of a nursing student whose body was found on a farm north of Glasgow have joined hundreds of mourners at a vigil. Karen Buckley vanished after a night out in the city at the weekend, sparking a four-day police search which yesterday found her body at High Craigton Farm. About 300 people - including the 24-year-old's mother and father Marian and John, and their son Kieran - attended a memorial at Glasgow's George Square today. A man, believed to be 21-year-old Alexander Pacteau, has been arrested in connection with Miss Buckley's death. He made a brief appearance in a private hearing at Glasgow Sheriff Court today. Scroll down for video . Grief stricken Marian and John Buckley (right) today joined hundreds of mourners at a vigil held in memory of their daughter Karen. Their son Kieran is pictured left . Miss Buckley was found dead on a farm north of Glasgow after going missing from a nightclub in the city . Miss Buckley's parents Marian and John (pictu